In [1]:
# Using Python 3.12.1 (local env: HeatNSalt)

# Process the OTU table of the repeated plate 6, starting with BOLD results. 

import pandas as pd
import numpy as np
import tiers

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
BOLD6_df = pd.read_excel('raw/ExStreamEPT6_apscale_OTUs_identification_result.xlsx', engine='openpyxl')

In [3]:
BOLD6_df.iloc[:,:14]

,id,Phylum,Class,Order,Family,Genus,Species,pct_identity,status,records,selected_level,BIN,flags,Seq
0,OTU_1,Arthropoda,Insecta,Ephemeroptera,Baetidae,Baetis,Baetis rhodani,100.0,public,77.0,Species,BOLD:AAM1760,NaN,TCTCGCCGCAAATATCGCCCACGGAGGGTCTTCGGTTGATTTCGCA...
1,OTU_2,Arthropoda,Insecta,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,100.0,private,40.0,Species,BOLD:AAC1503;BOLD:ADY7596;BOLD:AAF5737,2--5,CTTATCGAGTAATTTAGCTCATGCAGGAAGATCTGTTGATATTTCC...
2,OTU_3,Arthropoda,Insecta,Trichoptera,Goeridae,Goera,Goera pilosa,100.0,private,19.0,Species,BOLD:AAD4859,NaN,TTTATCAAGAAATTTAGCCCATGCAGGCAGTTCAGTTGATATTTCT...
3,OTU_4,Arthropoda,Insecta,Trichoptera,Leptoceridae,Athripsodes,Athripsodes cinereus,100.0,public,36.0,Species,BOLD:AAI7977,2,ATTAGCTAGCAATATTGCACATACAGGAAGCTCAGTTGATTTATCT...
4,OTU_5,Arthropoda,Insecta,Plecoptera,Nemouridae,Nemoura,Nemoura cinerea,100.0,private,96.0,Species,BOLD:AAG9376,NaN,CCTTTCTGCTGGTATTGCCCATGCTGGATCATCAGTAGACATAGCA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,OTU_2934,no-match,no-match,no-match,no-match,no-match,no-match,0.0,NaN,NaN,NaN,NaN,NaN,ATTAAGCAGTATTTCTGGCCACCCAGGTGGTGCCGTTGATTTAGGT...
2934,OTU_2935,Arthropoda,Insecta,Lepidoptera,Gelechiidae,Teleiodes,Teleiodes vulgella,100.0,public,16.0,Species,BOLD:AAE9855,2,TTTATCTTCTAATATTGCTCACGGAGGAAGTTCAGTTGATTTAGCA...
2935,OTU_2936,no-match,no-match,no-match,no-match,no-match,no-match,0.0,NaN,NaN,NaN,NaN,NaN,TTAGTGGAATCGAAGCGCACTCTGGCCCAGCCGTTGATTTCGGTAT...
2936,OTU_2937,Arthropoda,Insecta,Hymenoptera,Ichneumonidae,Stenomacrus,Stenomacrus laricis,100.0,public,3.0,Species,BOLD:ACT9253,2,ACTATCATTAAATATTAATCATGAAGGAATATCTGTTGATATAGCA...


In [ ]:
# Separate metadata / taxa info from read_counts
otu_info = BOLD6_df.iloc[:,:14]
read_matrix = BOLD6_df.set_index('id').iloc[:,14:]

# Extract OTU number for sorting
otu_info = otu_info.assign(idno=otu_info['id'].map(lambda x: x.split('_')[1]))
otu_info['idno'] = otu_info['idno'].astype(int)
otu_info = otu_info.sort_values('idno').reset_index(drop=True)

# Create a taxon hierarchy table
# Separate taxon information and add Kingdom value
otu_taxa_table = otu_info.iloc[:,:7].set_index('id')
otu_taxa_table.insert(loc=0, column='Kingdom', value='Animalia')

# set column names to lowercase
otu_taxa_table.columns = otu_taxa_table.columns.map(lambda x: x.lower())
otu_taxa_table = otu_taxa_table.assign(type=np.nan)
otu_taxa_table = otu_taxa_table.assign(label=tiers.get_leaves(otu_taxa_table))

# Merge to final otu_info table
otu_info = otu_taxa_table.merge(otu_info, left_index=True, right_on='id')

# Drop the old hierarchy before merging
otu_info = otu_info.drop(['Phylum', 'Class', 'Order', 'Family', 'Genus', 'species'],axis=1)

# Duplicates are removed from otu_taxa_table
otu_taxa_table = otu_taxa_table.drop_duplicates(keep='first').sort_values(otu_taxa_table.columns.tolist()).reset_index(drop=True)

In [5]:
read_matrix

,6_A10,6_A12,6_A2,6_A3,6_A4,6_A6,6_A7,6_A8,6_B1,6_B11,...,6_G9,6_H1,6_H10,6_H11,6_H3,6_H4,6_H5,6_H7,6_H8,6_H9
id,,,,,,,,,,,,,,,,,,,,,
OTU_1,0,2086,121,182,79,195,0,0,77,104,...,16502,89,82,91,58,159,57,27,10897,73
OTU_2,0,77,52,3388,3647,108,0,0,8808,7002,...,2,66,17,29,8202,3686,4,10,20,41
OTU_3,0,8,0,17,8,6,0,0,0,0,...,3,3,0,4558,0,7,3,9670,0,5
OTU_4,0,0,2973,0,0,1981,0,0,0,0,...,0,0,6,0,0,0,0,0,0,11
OTU_5,0,0,10,19,11,0,0,0,2,0,...,6,0,13617,0,0,40,6,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OTU_2934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OTU_2935,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
OTU_2936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:

# Handle read matrix
otu_sums = read_matrix.sum(axis=1)
any(otu_sums != 0)
# if True, Each OTU has some reads

# Function to get ordered list of non-zero values with indices
def non_zero_values(df):
    result = {}
    for col in df.columns:
        non_zero_values = df[col][df[col] != 0]  # Filter non-zero values
        if non_zero_values.empty:
            result[col] = []
            continue
        ordered_values = non_zero_values.sort_values()  # Sort by value
        result[col] = list(zip(ordered_values.index.astype(str), ordered_values))  # Store indices and values as list of tuples
    return result

ordered_non_zero_values = non_zero_values(read_matrix)

def replace_string_in_tuple(tup, df):
    id_value, reads = tup
    row = df.loc[df['id'] == id_value]

    # Check if the ID in the tuple matches with the ID in the DataFrame
    if not row.empty:
        # Find the corresponding label value from the DataFrame
        corresponding_label = row['label'].iloc[0]
        corresponding_species = row['Species'].iloc[0]
        corresponding_genus = row['genus'].iloc[0]
        corresponding_family = row['family'].iloc[0]
        corresponding_order = row['order'].iloc[0]
        # Replace the label value in the tuple
        return (id_value, corresponding_order, corresponding_family, corresponding_genus, corresponding_species, corresponding_label, reads)
    else:
        return tup  # If no matching ID is found, return the original tuple
    
species_dict = {}
count = 0
for sample, hits in ordered_non_zero_values.items():
    for hit in hits:
        species_dict[count] = [sample, replace_string_in_tuple(hit, otu_info)]
        count += 1

Megabarcoding_df = pd.DataFrame.from_dict(species_dict, orient='index')
Megabarcoding_df[['OTU', 'Order', 'Family', 'Genus', 'Species', 'Label', 'read count']] = pd.DataFrame(Megabarcoding_df[1].tolist(), index=Megabarcoding_df.index)
Megabarcoding_df = Megabarcoding_df.drop(columns=[1])
Megabarcoding_df = Megabarcoding_df.sort_values(by=[0, 'read count'], ascending=[True, False])

Megabarcoding_df = Megabarcoding_df.rename(columns={0: 'Sample'})
Megabarcoding_max_df = Megabarcoding_df.drop_duplicates(subset='Sample')

with pd.ExcelWriter('outputs/EPT_plate6_OTU_reads.xlsx') as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    Megabarcoding_max_df.to_excel(writer, sheet_name='Top Hits', index=False)
    Megabarcoding_df.to_excel(writer, sheet_name='All reads', index=False)


In [7]:
Megabarcoding_df_big = pd.read_excel('outputs/EPT_OTU_reads.xlsx')
merged = Megabarcoding_max_df.merge(Megabarcoding_df_big, on='Sample', suffixes=('_df1', '_df2'))
merged['Label_equal'] = merged['Label_df1'] == merged['Label_df2']

In [8]:
merged

,Sample,OTU_df1,Order_df1,Family_df1,Genus_df1,Species_df1,Label_df1,read count_df1,OTU_df2,Order_df2,Family_df2,Genus_df2,Species_df2,Label_df2,read count_df2,Label_equal
0,6_A12,OTU_1,Ephemeroptera,Baetidae,Baetis,Baetis rhodani,Baetis rhodani,2086,>OTU_5,Ephemeroptera,Baetidae,Baetis,Baetis rhodani,Baetis rhodani,83995,True
1,6_A2,OTU_4,Trichoptera,Leptoceridae,Athripsodes,Athripsodes cinereus,Athripsodes cinereus,2973,>OTU_7,Trichoptera,Leptoceridae,Athripsodes,Athripsodes cinereus,Athripsodes cinereus,44861,True
2,6_A3,OTU_2,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,3388,>OTU_1,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,111047,True
3,6_A4,OTU_2,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,3647,>OTU_1,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,136786,True
4,6_A6,OTU_4,Trichoptera,Leptoceridae,Athripsodes,Athripsodes cinereus,Athripsodes cinereus,1981,>OTU_7,Trichoptera,Leptoceridae,Athripsodes,Athripsodes cinereus,Athripsodes cinereus,67404,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,6_H4,OTU_2,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,3686,>OTU_1,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,103224,True
57,6_H5,OTU_8,Trichoptera,Limnephilidae,Potamophylax,Potamophylax rotundipennis,Potamophylax rotundipennis,11013,>OTU_3,Trichoptera,Limnephilidae,Potamophylax,Potamophylax rotundipennis,Potamophylax rotundipennis,223222,True
58,6_H7,OTU_3,Trichoptera,Goeridae,Goera,Goera pilosa,Goera pilosa,9670,>OTU_2,Trichoptera,Goeridae,Goera,Goera pilosa,Goera pilosa,197338,True
59,6_H8,OTU_1,Ephemeroptera,Baetidae,Baetis,Baetis rhodani,Baetis rhodani,10897,>OTU_5,Ephemeroptera,Baetidae,Baetis,Baetis rhodani,Baetis rhodani,251863,True


In [9]:
differences = merged[merged['Label_df1'] != merged['Label_df2']]
differences

,Sample,OTU_df1,Order_df1,Family_df1,Genus_df1,Species_df1,Label_df1,read count_df1,OTU_df2,Order_df2,Family_df2,Genus_df2,Species_df2,Label_df2,read count_df2,Label_equal
14,6_B8,OTU_9,Odonata,Coenagrionidae,Ischnura,Ischnura elegans,Ischnura elegans,548,>OTU_22,Odonata,Coenagrionidae,Ischnura,Ischnura genei,Ischnura genei,14228,False
15,6_B9,OTU_116,Harpacticoida,Canthocamptidae,Canthocamptus,Canthocamptus staphylinus,Canthocamptus staphylinus,53,>OTU_2,Trichoptera,Goeridae,Goera,Goera pilosa,Goera pilosa,147316,False
25,6_D11,OTU_10,Diptera,Chironomidae,NaN,NaN,Chironomidae,633,>OTU_4,Ephemeroptera,Ephemeridae,Ephemera,Ephemera danica,Ephemera danica,19051,False
30,6_D6,OTU_10,Diptera,Chironomidae,NaN,NaN,Chironomidae,786,>OTU_22,Odonata,Coenagrionidae,Ischnura,Ischnura genei,Ischnura genei,23040,False
42,6_F3,OTU_7,Diptera,Chironomidae,Rheocricotopus,Rheocricotopus fuscipes,Rheocricotopus fuscipes,448,>OTU_21,Diptera,Chironomidae,NaN,NaN,Chironomidae,14865,False
45,6_F7,OTU_7,Diptera,Chironomidae,Rheocricotopus,Rheocricotopus fuscipes,Rheocricotopus fuscipes,649,>OTU_22,Odonata,Coenagrionidae,Ischnura,Ischnura genei,Ischnura genei,20467,False
50,6_G8,OTU_7,Diptera,Chironomidae,Rheocricotopus,Rheocricotopus fuscipes,Rheocricotopus fuscipes,378,>OTU_4,Ephemeroptera,Ephemeridae,Ephemera,Ephemera danica,Ephemera danica,23519,False
52,6_H1,OTU_11,Diptera,Dolichopodidae,Sciapus,Sciapus wiedemanni,Sciapus wiedemanni,297,>OTU_4,Ephemeroptera,Ephemeridae,Ephemera,Ephemera danica,Ephemera danica,14480,False
